In [1]:
import findspark
findspark.init()
import pyspark
import re

import numpy as np
import scipy.sparse as sps
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.regression import LabeledPoint

In [2]:
import os
port = 4040 + hash(os.getcwd().split("/")[2])%200
config = pyspark.SparkConf().set('spark.executor.memory', '512M')\
         .set('spark.cores.max', '2')\
         .set('spark.port.maxRetries','200')\
         .set('spark.ui.port', port)    
print port,     os.getcwd().split("/")[2]

4182 zhl617


In [3]:
sc = pyspark.SparkContext(appName="Simrank ingredient", master='spark://polyp1:7077',conf = config )

In [4]:
rddData = sc.textFile("/scratch/ISE495/data/recipes.txt")

To make your life easier, the following 4 dictionaries has been provided:

cuisine_encoder: each cuisine is given a index

ingredient_encoder: each ingredient is given a index

cuisine_counter: how many recipes are there in each cuisine

ingredient_counter: how many times each ingredient appear in the whole data

In [33]:
cuisine_encoder = np.load('/scratch/ISE495/data/catogDic.npy').item()

cuisine_counter = np.load('/scratch/ISE495/data/catogCount.npy').item()

diclist=rddData.flatMap(lambda x: x.split(";",2)[2].split(";")).distinct().collect()
ingredient_encoder ={f:i for i, f in enumerate(diclist)}

ingredientlist=rddData.flatMap(lambda x:x.split(";",2)[2].split(";")).map(lambda x: (x,1)).reduceByKey(lambda x,y:x+y).collect()
ingredient_counter = {ingredient_encoder[i]:f for i,f in ingredientlist}

In [6]:
print cuisine_encoder
print "\n\n\n"
print cuisine_counter


{u'irish': 0, u'mexican': 1, u'chinese': 2, u'filipino': 3, u'vietnamese': 13, u'spanish': 14, u'japanese': 15, u'british': 7, u'french': 5, u'greek': 16, u'indian': 17, u'jamaican': 6, u'moroccan': 4, u'brazilian': 18, u'russian': 8, u'cajun_creole': 9, u'thai': 10, u'southern_us': 11, u'korean': 12, u'italian': 19}




{0: 667, 1: 6438, 2: 2673, 3: 755, 4: 821, 5: 2646, 6: 526, 7: 804, 8: 489, 9: 1546, 10: 1539, 11: 4320, 12: 830, 13: 825, 14: 989, 15: 1423, 16: 1175, 17: 3003, 18: 467, 19: 7838}
{u'condensedsoup': 1749, u'frozenspinach': 2306, u'condensedcheddarcheesesoup': 3686, u'citrusfruit': 2631, u'freshdates': 190, u'orangeglaze': 6619, u'wholewheatpennepasta': 2891, u'backbaconrashers': 2582, u'mackerel': 21, u'sliveredalmonds': 2803, u'honeygrahamcrackers': 777, u'sesameseeds': 2468, u'nonfatsweetenedcondensedmilk': 5734, u'figs': 3381, u'corngrits': 3868, u'groundtumeric': 2475, u'seedlessraspberryjam': 1363, u'jamaicanjerkmarinade': 5709, u'caramels': 238, u'broiler': 35, 

### Use SimRank idea to find out which cuisine is most similar to thai cuisine.

put the place of thai to 1 and others to 0

In [34]:
X=np.zeros(20)
X[cuisine_encoder["thai"]]=1

In [35]:
linkData = rddData.map(lambda x:x.split(';'))
linkData = linkData.map(lambda x: (int(cuisine_encoder[x[0]]), (int(x[1]),([ingredient_encoder[i] for i in x[2:]])))).sortBy(lambda x:x[0])

In [10]:
linkData.take(1)

[(0, (33755, [373, 310, 53, 531, 1258, 115, 2193, 964, 27, 1913, 1530, 11]))]

SamRank Algorithm

In [38]:
xNew=X

beta=0.8
for i in xrange(10):
    def gg(y):
        result = []
        for i in y[1][1]:
            result.append((i,xNew[y[0]]/len(y[1][1])/cuisine_counter[y[0]]))
        return result
    newy = linkData.flatMap(gg).reduceByKey(lambda x,y:x+y).sortBy(lambda x:x[0]).collect()

    def kk(y):
        result1 = []
        for i in y[1][1]:
            result1.append((y[0],newy[i][1]/ingredient_counter[i]))
        return result1
   
    xc=linkData.flatMap(kk).reduceByKey(lambda x,y:x+y).sortBy(lambda x:x[0]).collect()

    xNew=[i[1] for i in xc]
    xsum=sum(xNew)
    
    xNew= (1-beta)/xsum*np.array(xNew)+beta*X
    print xNew



[0.00127739 0.02483342 0.02381732 0.00518642 0.0034112  0.00594684
 0.00309743 0.00158679 0.00114494 0.00523611 0.84109911 0.01042529
 0.00611715 0.01397185 0.00293365 0.00779805 0.00307872 0.01881097
 0.00238718 0.01784014]
[0.00148823 0.02595105 0.02319848 0.00502579 0.00360477 0.00668114
 0.00309438 0.00186383 0.00131059 0.005694   0.83614238 0.01161243
 0.00605747 0.01276307 0.0031559  0.00790165 0.00337248 0.01888111
 0.00236874 0.0198325 ]
[0.00150416 0.0260094  0.0230687  0.00500526 0.00361527 0.0067452
 0.0030918  0.00188499 0.0013214  0.0057275  0.83593188 0.01170914
 0.0060273  0.01267949 0.00317329 0.00787268 0.00339607 0.01884393
 0.00236578 0.02002678]
[0.00150531 0.02601308 0.02305592 0.0050036  0.00361592 0.00675043
 0.00309157 0.0018865  0.00132212 0.00572997 0.83592051 0.01171648
 0.00602394 0.01267423 0.00317468 0.00786843 0.00339798 0.01883956
 0.00236564 0.02004412]
[0.00150539 0.02601335 0.02305483 0.00500347 0.00361596 0.00675085
 0.00309155 0.00188661 0.00132217 

In [32]:
for i in sorted(zip(xNew,range(len(xNew))),reverse=True):
    print cuisine_encoder.keys() [list (cuisine_encoder.values()).index (i[1])]

thai
mexican
chinese
italian
indian
vietnamese
southern_us
japanese
french
korean
cajun_creole
filipino
moroccan
greek
spanish
jamaican
brazilian
british
irish
russian


In [ ]:
sc.stop()